# Implement RRT and its variant on UR5

In [1]:
import example_robot_data as robex
import hppfcl
import math
import numpy as np
import pinocchio as pin
import time
from tqdm import tqdm
from typing import Union, List, Tuple, Callable

In [2]:
import matplotlib.pylab as plt; plt.ion()

In [3]:
from utils.meshcat_viewer_wrapper import MeshcatVisualizer, colors
from utils.datastructures.storage import Storage
from utils.datastructures.pathtree import PathTree
from utils.datastructures.mtree import MTree
from utils.collision_wrapper import CollisionWrapper
from pinocchio.robot_wrapper import RobotWrapper

## Load UR5

In [4]:
robot = robex.load('ur5')
collision_model = robot.collision_model
visual_model = robot.visual_model

Recall some placement for the UR5

In [5]:
a = robot.placement(robot.q0, 6)  # Placement of the end effector joint.
b = robot.framePlacement(robot.q0, 22)  # Placement of the end effector tip.

tool_axis = b.rotation[:, 2]  # Axis of the tool
tool_position = b.translation

In [6]:
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [7]:
# viz.viewer.jupyter_cell()

Set a start and a goal configuration

In [11]:
q_i = np.array([1., -1.5, 2.1, -.5, -.5, 0])
q_g = np.array([3., -1., 1, -.5, -.5, 0])
radius = 0.05

In [12]:
viz.display(q_i)
M = robot.framePlacement(q_i, 22)
name = "world/sph_initial"
viz.addSphere(name, radius, [0., 1., 0., 1.])
viz.applyConfiguration(name,M)

In [13]:
viz.display(q_g)
M = robot.framePlacement(q_g, 22)
name = "world/sph_goal"
viz.addSphere(name, radius, [0., 0., 1., 1.])
viz.applyConfiguration(name,M)

## Implement everything needed for RRT

We abstract the robot the environment and its behaviour in a class call `System`

It must be able to:
- generate random configuration which are not colliding if needed (sampling)
- implement a distance on the configuration space (distance)
- generate path between two configuration (steering)
- check if a path is free between two configuration and return the latest free config (directional free steering)
and some function to display the configuration.

Recall that in the case of the UR5 the configuration space is $S_1^{6}$, where $S_1$ is the unit cirle, we can parametrize by $\theta\in[-\pi,\pi]$ such that $-\pi$ and $\pi$ are identified.

In the next cell, you must implement the system behaviour for the UR5.

In [17]:
class System():
    
    def __init__(self, robot: RobotWrapper):
        self.robot = robot
        robot.gmodel = robot.collision_model
        self.display_edge_count = 0
        self.colwrap = CollisionWrapper(robot)  # For collision checking
        self.nq = self.robot.nq
        self.display_count = 0
        self.dof = 6 # degrees of freedom
    
    @staticmethod
    def distance(q1: np.ndarray, q2: np.ndarray) -> Union[float, np.array]:
        """
        Distance between q1 and q2 in the config space
        supports batch of config.
        """
        if len(q2.shape) > len(q1.shape):
            q1 = q1[None, ...]
        
        e = np.mod(np.abs(q1 - q2), 2 * np.pi)
        e[e > np.pi] = 2 * np.pi - e[e > np.pi]
        return np.linalg.norm(e, axis=-1)

    def random_config(self, free=True) -> np.ndarray:
        """
        Return a random configuration which is not in collision if free=True
        Does not set the robot in this configuration
        """
        q = 2 * np.pi * np.random.rand(self.dof) - np.pi  # [-pi, pi]^6
        if not free:
            return q
        while self.is_colliding(q):
            q = 2 * np.pi * np.random.rand(self.dof) - np.pi
        return q

    def is_colliding(self, q: np.array) -> bool:
        """
        Uses CollisionWrapper to decide if a configuration is in collision
        """
        # @TODO: are self collisions handled here
        self.colwrap.computeCollisions(q)
        collisions = self.colwrap.getCollisionList()
        return (len(collisions) > 0)

    def get_path(self, q1: np.array, q2: np.array, l_min=None, l_max=None, eps: float=0.2) -> np.ndarray:
        """Generate a straight continuous path (in the config space) between q1 and q2
        with precision eps (in radians) between q1 and q2 
        If l_min or l_max is mentioned, extrapolate or cut the path such
            - if l_min > d , extrapolate path_length=l_min
            - if l_max < d, cut path_length=l_max
        otherwise, if l_min<d(q1,q2)<l_max , path_length=d(q1,q2)
        
        ```

                         cut l_max <d         standard    extrapolate l_min>d
                              |                  |             |
                              v                  v             v
        [q1, q2] -> [q1, ... , ... ,... , ... ,  q2]    ........
                        <->
                        eps
        ```
        """
        q1 = np.mod(q1 + np.pi, 2 * np.pi) - np.pi
        q2 = np.mod(q2 + np.pi, 2 * np.pi) - np.pi

        diff = q2 - q1
        query = np.abs(diff) > np.pi
        q2[query] = q2[query] - np.sign(diff[query]) * 2 * np.pi

        d = self.distance(q1, q2)
        if d < eps:
            # precision is higher than the provided segment
            # return the original segment (q1 & q2 endpoints, nothing in-between)
            return np.stack([q1, q2], axis=0)
        
        if l_min is not None or l_max is not None:
            new_d = np.clip(d, l_min, l_max)
        else:
            new_d = d
            
        N = int(new_d / eps + 2)

        return np.linspace(q1, q1 + (q2 - q1) * new_d / d, N)
        
    def is_free_path(self, q1: np.ndarray, q2: np.ndarray, l_min=0.2, l_max=1., eps=0.2) -> Tuple[bool, np.ndarray]:
        """
        Create a path and check collision to return the last
        non-colliding configuration.
        Return X, q  -> q is the last state before collision, 
        in a max range of length l_max (steer a unitary vector in the diretion of q2)
        if no obstacle in the way, we shall have q= q1 + l_max*(q2-q1)/|q2-q1|
        where X is a boolean which state if the steering has worked.
        We require at least l_min must be cover without collision to validate the path.
        """
        q_path = self.get_path(q1, q2, l_min, l_max, eps)
        N = len(q_path)
        N_min = N - 1 if l_min is None else min(N - 1, int(l_min / eps))
        for i in range(N):
            if self.is_colliding(q_path[i]):
                break
        if i < N_min:
            return False, None
        if i == N - 1:
            return True, q_path[-1] # return the last point of the path
        return True, q_path[i - 1] # return the last point before collision
    
    # --------------------------------------------------------------------------------
    # Visualization functions
    # --------------------------------------------------------------------------------
    def reset(self):
        """
        Reset the system visualization
        """
        for i in range(self.display_count):
            viz.delete(f"world/sph{i}")
            viz.delete(f"world/cil{i}")
        self.display_count = 0
    
    def display_edge(self, q1: np.ndarray, q2: np.ndarray, radius=0.01, color=[1.,0.,0.,1]):
        """Visualize an edge"""
        M1 = self.robot.framePlacement(q1, 22)  # Placement of the end effector tip.
        M2 = self.robot.framePlacement(q2, 22)  # Placement of the end effector tip.
        middle = .5 * (M1.translation + M2.translation)
        direction = M2.translation - M1.translation
        length = np.linalg.norm(direction)
        dire = direction / length
        orth = np.cross(dire, np.array([0, 0, 1]))
        orth2 = np.cross(dire, orth)
        Mcyl = pin.SE3(np.stack([orth2, dire, orth], axis=1), middle)
        name = f"world/sph{self.display_count}"
        viz.addSphere(name, radius, [1.,0.,0.,1])
        viz.applyConfiguration(name,M2)
        name = f"world/cil{self.display_count}"
        viz.addCylinder(name, length, radius / 4, [0., 1., 0., 1])
        viz.applyConfiguration(name,Mcyl)
        self.display_count +=1
        
    def display_motion(self, qs: np.ndarray, step=1e-1):
        # Given a point path display the smooth movement
        for i in range(len(qs) - 1):
            for q in self.get_path(qs[i], qs[i+1])[:-1]:
                viz.display(q)
                time.sleep(step)
        viz.display(qs[-1])


In [18]:
system = System(robot)

In [19]:
system.distance(np.array([np.pi-0.1]), np.array([[-np.pi+0.1], [0.]]))
system.distance(np.array([[-np.pi+0.1], [0.]]), np.array([[np.pi-0.1]]))

system.distance(np.array([-np.pi+0.1, 0.]), np.array([[np.pi-0.1, 0.2], [np.pi-0.1, -0.1]]))
system.distance(np.array([[np.pi-0.1, 0.2], [np.pi-0.1, -0.1]]), np.array([-np.pi+0.1, 0.]))

array([0.28284271, 0.2236068 ])

In [20]:
system.distance(q_i, q_g)
system.distance(np.array([q_i, q_i]), np.array([q_i, q_g]))

array([0.        , 2.33666429])

In [22]:
system.display_motion(system.get_path(q_i, q_g))

## RRT implementation

In its most simple form, RRT construct a tree from the start, eventually with a bias toward the goal. In the following class, we add some memoization to avoid recomputing distances. The kNN (k Nearest Neighbors) structure works on node indices.

Let us look at an implementation the core algorithm:

In [23]:
class RRT():
    """
    Can be splited into RRT base because different rrt
    have factorisable logic
    """
    def __init__(
        self,
        system: System,
        node_max: int = 500000,
        iter_max: int = 1000000,
        N_bias: int=10,
        l_min: float =.2,
        l_max: float =.5,
        steer_delta: float=.1,
    ) -> None:
        '''Class implementing the RRT (Rapidly exploring Random Tree) algorithm.
        Creates a tree (a special kind of graph)
        starting from the source
        and progressively sampling free paths until reaching the target.

        Args:
            system (System): contains the Robotwrapper and a few helpers to perform computations in the configuration space.
            node_max (int, optional): maximum number of nodes in the tree. Defaults to 500000.
            iter_max (int, optional): maximum number of iterations. Defaults to 1000000.
            N_bias (int, optional): _description_. Defaults to 10.
            l_min (float, optional): _description_. Defaults to .2.
            l_max (float, optional): _description_. Defaults to .5.
            steer_delta (float, optional): _description_. Defaults to .1.
        '''
        self.system = system
        # params
        self.l_max = l_max
        self.l_min = l_min
        self.N_bias = N_bias
        self.node_max = node_max
        self.iter_max = iter_max
        self.steer_delta = steer_delta
        # intern
        self.NNtree = None
        self.storage = None
        self.pathtree = None
        # The distance function will be called on N, dim object
        self.real_distance = self.system.distance
        # Internal for computational_opti in calculating distance
        self._candidate = None
        self._goal = None
        self._cached_dist_to_candidate = {}
        self._cached_dist_to_goal = {}

    def distance(self, q1_idx: int, q2_idx: int) -> float:
        """Compute the real distance
        ?Along the tree of the feasible path between q1 & q2
        (not the naïve straight line euclian distance in the configuration space)

        Args:
            q1_idx (int): start node index (in the tree)
            q2_idx (int): end node index (in the tree)

        Returns:
            float: distance
        """
        if isinstance(q2_idx, int):
            if q1_idx == q2_idx:
                return 0. # requesting twice the same node -> 0
            if q1_idx == -1 or q2_idx == -1:
                if q2_idx == -1:
                    q1_idx, q2_idx = q2_idx, q1_idx
                if q2_idx not in self._cached_dist_to_candidate:
                    self._cached_dist_to_candidate[q2_idx] = self.real_distance(
                        self._candidate, self.storage[q2_idx]
                    )
                return self._cached_dist_to_candidate[q2_idx]
            if q1_idx == -2 or q2_idx == -2:
                if q2_idx == -2:
                    q1_idx, q2_idx = q2_idx, q1_idx
                if q2_idx not in self._cached_dist_to_goal:
                    self._cached_dist_to_goal[q2_idx] = self.real_distance(
                        self._goal, self.storage[q2_idx]
                    )
                return self._cached_dist_to_goal[q2_idx]
            return self.real_distance(self.storage[q1_idx], self.storage[q2_idx])
        if q1_idx == -1:
            q = self._candidate
        elif q1_idx == -2:
            q = self._goal
        else:
            q = self.storage[q1_idx]
        return self.real_distance(q, self.storage[q2_idx])

    def new_candidate(self) -> np.ndarray:
        """Sample a candidate in the free space. 
        Init self._candidate, reset _cached_dist_to_candidate
        """
        q = self.system.random_config(free=True)
        self._candidate = q
        self._cached_dist_to_candidate = {}
        return q

    def solve(self, qi: np.ndarray, validate: Callable, qg: np.ndarray=None) -> bool:
        """Run the algorithm

        Args:
            qi (np.ndarray): initial configuration
            validate (Callable): function which returns 
            qg (np.ndarray, optional): goal/target configuration. 
            Defaults to None (validate may be enough to contain the fact of reaching the target).
            Goal is required when using the N_bias

        Returns:
            bool: sucess
        """
        self.system.reset()
        self._goal = qg
        
        # Reset internal datastructures
        self.storage = Storage(self.node_max, self.system.nq)
        self.pathtree = PathTree(self.storage) # to store the tree of the path only
        self.NNtree = MTree(self.distance) # to store the exploration tree
        # Root of the tree = initial state qi
        qi_idx = self.storage.add_point(qi)
        self.NNtree.add_point(qi_idx)
        self.it_trace = []

        found = False
        iterator = range(self.iter_max)
        for i in tqdm(iterator):
            # Sample new candidate
            if i % self.N_bias == 0: 
                # Every N_bias times, let's force the candadidate to the goal!
                q_new = self._goal
                q_new_idx = -2
            else:
                # Standard random sampling
                q_new = self.new_candidate()
                q_new_idx = -1

            # Find closest neighboor to q_new in the NNtree
            q_near_idx, d = self.NNtree.nearest_neighbour(q_new_idx) # q_new_idx = -1 (or -2 in case of bias in favor of the target).
            
            # Steer from it toward the new checking for colision, with  a max length of lmax.
            # q_prox is a single feasible state (not the whole segment)
            success, q_prox = self.system.is_free_path(
                self.storage.data[q_near_idx],
                q_new,
                l_min=self.l_min,
                l_max=self.l_max,
                eps=self.steer_delta
            )

            if not success:
                self.it_trace.append(0)
                continue
            self.it_trace.append(1)
            
            # Add the q_prox point as new node in data structure
            q_prox_idx = self.storage.add_point(q_prox) # add the q_prox point to the storage, get new index
            self.NNtree.add_point(q_prox_idx) # add the node q_prox to the graph
            self.pathtree.update_link(q_prox_idx, q_near_idx)
            self.system.display_edge(self.storage[q_near_idx], self.storage[q_prox_idx])

            # Test if it reach the goal
            if validate(q_prox):
                q_g_idx = self.storage.add_point(q_prox)
                self.NNtree.add_point(q_g_idx)
                self.pathtree.update_link(q_g_idx, q_prox_idx)
                found = True
                break
        self.iter_done = i + 1
        self.found = found
        return found

    def get_path(self, q_g):
        assert self.found
        path = self.pathtree.get_path()
        return np.concatenate([path, q_g[None, :]])


In proper code, we would document the parameters of our functions.

- **Your turn:** Add docstrings to the code above, following the [Google style](https://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html).
- Optional: you are welcome to add type annotations if you'd like.

The constructor of the `RRT` class invites you to start.

For this problem, we will instantiate our RRT with the following parameters:

In [24]:
rrt = RRT(
    system,
    N_bias=20,
    l_min=0.2,
    l_max=0.5,
    steer_delta=0.1,
)

Now let's define our termination condition, and run the main function:

In [27]:
eps_final = .1
def validation(key):
    vec = robot.framePlacement(key, 22).translation - robot.framePlacement(q_g, 22).translation
    return (float(np.linalg.norm(vec)) < eps_final)

rrt.solve(q_i, validation, qg=q_g)

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 60/1000000 [00:03<15:48:40, 17.57it/s]


True

In [26]:
system.display_motion(rrt.get_path(q_g))

In [ ]:
system.reset()

## Create obstacle with environments

We already had some simple algorithms to find free paths, *i.e.* without obstacles. Let us now add some obstacles to the environment:

In [28]:
robot = robex.load('ur5')
collision_model = robot.collision_model
visual_model = robot.visual_model

In [29]:
def addCylinderToUniverse(name, radius, length, placement, color=colors.red):
    geom = pin.GeometryObject(
        name,
        0,
        hppfcl.Cylinder(radius, length),
        placement
    )
    new_id = collision_model.addGeometryObject(geom)
    geom.meshColor = np.array(color)
    visual_model.addGeometryObject(geom)
    
    for link_id in range(robot.model.nq):
        collision_model.addCollisionPair(
            pin.CollisionPair(link_id, new_id)
        )
    return geom

In [30]:
from pinocchio.utils import rotate

[collision_model.removeGeometryObject(e.name) for e in collision_model.geometryObjects if e.name.startswith('world/')]

# Add a red box in the viewer
radius = 0.1
length = 1.

cylID = "world/cyl1"
placement = pin.SE3(pin.SE3(rotate('z',np.pi/2), np.array([-0.5,0.4,0.5])))
addCylinderToUniverse(cylID,radius,length,placement,color=[.7,.7,0.98,1])


cylID = "world/cyl2"
placement = pin.SE3(pin.SE3(rotate('z',np.pi/2), np.array([-0.5,-0.4,0.5])))
addCylinderToUniverse(cylID,radius,length,placement,color=[.7,.7,0.98,1])

cylID = "world/cyl3"
placement = pin.SE3(pin.SE3(rotate('z',np.pi/2), np.array([-0.5,0.7,0.5])))
addCylinderToUniverse(cylID,radius,length,placement,color=[.7,.7,0.98,1])


cylID = "world/cyl4"
placement = pin.SE3(pin.SE3(rotate('z',np.pi/2), np.array([-0.5,-0.7,0.5])))
addCylinderToUniverse(cylID,radius,length,placement,color=[.7,.7,0.98,1])

In [31]:
q_i = np.array([-1., -1.5, 2.1, -.5, -.5, 0])
q_g = np.array([3.1, -1., 1, -.5, -.5, 0])
radius = 0.05

We need to reload the viewer

In [32]:
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [33]:
viz.display(q_i)
M = robot.framePlacement(q_i, 22)
name = "world/sph_initial"
viz.addSphere(name, radius, [0., 1., 0., 1.])
viz.applyConfiguration(name,M)

In [34]:
viz.display(q_g)
M = robot.framePlacement(q_g, 22)
name = "world/sph_goal"
viz.addSphere(name, radius, [0., 0., 1., 1.])
viz.applyConfiguration(name,M)

In [35]:
viz.display(q_g)

In [36]:
system = System(robot)

In [37]:
rrt = RRT(
    system,
    N_bias=20,
    l_min=0.2,
    l_max=0.5,
    steer_delta=0.1,
)

In [38]:
eps_final = .1

def validation(key):
    vec = robot.framePlacement(key, 22).translation - robot.framePlacement(q_g, 22).translation
    return (float(np.linalg.norm(vec)) < eps_final)

rrt.solve(q_i, validation, qg=q_g)

  0%|          | 736/1000000 [01:14<28:13:50,  9.83it/s]


True

In [39]:
system.display_motion(rrt.get_path(q_g))

And solve RRT. It is long right ? Let us implement more efficient algorithms


![](figures/RRT.png)

## Bi-RRT

Now it's your turn. Make a `BiRRT` class, similar to the `RRT` class above, but implementing the Bi-RRT algorithm. (It is not recommended to try to inherit from `RRT`, as you will end up re-implementing most functions.) Here is a template you are free to adapt, with some advice:

In [ ]:
class BiRRT(RRT):
    def __init__(
        self,
        system,
        node_max=500000,
        iter_max=1000000,
        l_min=.2,
        l_max=.5,
        steer_delta=.1,
    ):
        # Initialize attributes:
        # self.l_min = l_min
        # etc.

        # New: duplicate this attribute as dictionaries with two keys:
        # "forward" and "backward". See `solve` below.
        self._cached_dist_to_candidate = {}
        self.storage = {}
        self.pathtree = {}
        self.tree = {}

    def tree_distance(self, direction: str, q1_idx, q2_idx):
        # Adapt from RRT.distance
        # There is now a direction string to select the underlying tree,
        # either "forward" (from q_init) or "backward" (from q_goal).

    def forward_distance(self, q1_idx, q2_idx):
        return self.tree_distance("forward", q1_idx, q2_idx)

    def backward_distance(self, q1_idx, q2_idx):
        return self.tree_distance("backward", q1_idx, q2_idx)

    def new_candidate(self):
        # A minor change is required to adapt RRT.new_candidate to this template.

    def solve(self, qi, validate, qg=None):
        # Reset internal datastructures
        for direction in ("forward", "backward"):
            self._cached_dist_to_candidate[direction] = {}
            self.storage[direction] = Storage(node_max, system.nq)
            self.pathtree[direction] = PathTree(self.storage[direction])
        self.tree = {
            "forward": MTree(self.forward_distance),
            "backward": MTree(self.backward_distance),
        }

        # Now datastructures are initialized
        # The rest is up to you! 

    def get_path(self):
        assert self.found
        forward_path = self.pathtree["forward"].get_path()
        backward_path = self.pathtree["backward"].get_path()
        return np.concatenate([forward_path, backward_path[::-1]])

You should be able to call `BiRRT` similarly to `RRT`:

In [ ]:
system.reset()

birrt = BiRRT(
    system,
    l_min=0.2,
    l_max=0.5,
    steer_delta=0.1,
)

birrt.solve(q_i, validation, qg=q_g)

In [ ]:
system.display_motion(birrt.get_path())

How many iterations did it take to find a solution? Is it faster than previously with `RRT`?

## Bonus question: Bi-RRT*

Implement an optimal variant `BiRRTStar` of your `BiRRT` class and run it in the same configuration as the two algorithms above. What do you notice about the resulting tree? What is the improvement in overall path length between `RRT`, `BiRRT` and `BiRRTStar`?